In [1]:
import pandas as pd
df = pd.read_csv('jira_tickets_cache.csv')
df.head(2)

,ticket_id,summary,description,status,priority,reporter,assignee,created_at,updated_at,labels,components,owned_by_team,brand,product,geo_region,environment,root_cause,sprint,comments,url
0,CP-2619,[Optum] Cleanup to fix issues due to incorrect...,"Due to incorrect config in the Stride feature,...",Open,Highest-P0,Prem Rajiv,NaN,2025-04-26T16:55:28.693+0530,2025-05-06T17:27:14.402+0530,"[""optumcoreplatformissue""]","[""Loyalty""]",NaN,"[""Abbott_HK_Prod""]",NaN,NaN,"[""Prod""]",NaN,NaN,"[{""author"": ""Harish Kumar"", ""timestamp"": ""2025...",NaN
1,CP-490,Trigger Event Replay for City of Tampa Februar...,For the promotions that were created post Febr...,Open,Medium-P2,Prem Rajiv,NaN,2025-04-26T13:02:48.563+0530,2025-05-06T17:27:11.657+0530,"[""optumcoreplatformissue""]","[""Loyalty""]",NaN,"[""Abbott_HK_Prod""]",NaN,NaN,"[""Prod""]",NaN,NaN,"[{""author"": ""Harish Kumar"", ""timestamp"": ""2025...",NaN


In [2]:
import pandas as pd
import re # We'll need this for regex operations later

# --- Step 1 & 2 for 'summary' and 'description' ---

# IMPORTANT: Verify these column names match your DataFrame!
summary_col = 'summary'
description_col = 'description'
comments_col = 'comments' # We'll use this later

# Create new columns for cleaned text to keep originals safe
# Fill NaN (missing) values with an empty string before any cleaning
df['cleaned_summary'] = df[summary_col].fillna('').astype(str)
df['cleaned_description'] = df[description_col].fillna('').astype(str)

# Let's display the first few rows of original and new columns to see the effect
print("Original and initial cleaned 'summary' and 'description':")
print(df[[summary_col, 'cleaned_summary', description_col, 'cleaned_description']].head())

print(f"\nMissing values in 'cleaned_summary': {df['cleaned_summary'].isnull().sum()}")
print(f"Missing values in 'cleaned_description': {df['cleaned_description'].isnull().sum()}")

Original and initial cleaned 'summary' and 'description':
                                             summary  \
0  [Optum] Cleanup to fix issues due to incorrect...   
1  Trigger Event Replay for City of Tampa Februar...   
2    Optum - Promotion rule not evaluating correctly   
3                  Request for Access to Dev Console   
4    Access to AWS Workspace and API tester for USHC   

                                     cleaned_summary  \
0  [Optum] Cleanup to fix issues due to incorrect...   
1  Trigger Event Replay for City of Tampa Februar...   
2    Optum - Promotion rule not evaluating correctly   
3                  Request for Access to Dev Console   
4    Access to AWS Workspace and API tester for USHC   

                                         description  \
0  Due to incorrect config in the Stride feature,...   
1  For the promotions that were created post Febr...   
2  We have a case where a promotion rule appears ...   
3  Hi Team,\nAccess to the dev console has b

In [8]:
import json
import pandas as pd

# --- Step 1 & 2 for 'comments' (UPDATED with Timestamp Sorting) ---

comments_col = 'comments'

def parse_and_format_comments(comments_json_str):
    """
    Parses a JSON string of comments, sorts them by timestamp, 
    extracts author and cleaned_body, formats them, and joins them.
    """
    if pd.isna(comments_json_str) or not comments_json_str.strip():
        return ""

    try:
        comments_list = json.loads(comments_json_str)
        if not isinstance(comments_list, list) or not comments_list:
            return ""

        # --- Sort comments by timestamp (oldest to newest) ---
        # Assuming timestamp is in a format that allows direct string comparison (like ISO 8601)
        # If timestamps are complex, they might need to be parsed to datetime objects first,
        # but for ISO 8601 format 'YYYY-MM-DDTHH:MM:SS.mmm+ZZZZ', string sort is fine.
        try:
            comments_list.sort(key=lambda x: x.get('timestamp', ''))
        except TypeError:
            # Fallback if timestamps are not consistently strings or are missing in some objects,
            # though this is less likely if the data is well-formed.
            # print(f"Warning: Could not sort comments by timestamp for: {str(comments_json_str)[:100]}...")
            pass # Proceed with original order if sorting fails
        # --- End of sorting ---

        formatted_comments = []
        for comment_obj in comments_list:
            author_name = "Unknown Author"
            if 'author' in comment_obj and isinstance(comment_obj['author'], str):
                author_name = comment_obj['author']
            elif isinstance(comment_obj.get('author'), dict):
                author_name = comment_obj['author'].get('displayName', comment_obj['author'].get('name', "Unknown Author"))

            comment_text = comment_obj.get('cleaned_body', '') 

            if comment_text:
                formatted_comments.append(f"{author_name}: {comment_text}")
        
        return "\n".join(formatted_comments)

    except json.JSONDecodeError:
        # print(f"Warning: Could not decode JSON for comments: {str(comments_json_str)[:100]}...")
        return ""
    except Exception as e:
        # print(f"Warning: Error processing comments '{str(comments_json_str)[:100]}...': {e}")
        return ""

# Apply the function to the comments column
df['cleaned_comments'] = df[comments_col].apply(parse_and_format_comments)

# Display the first few rows of original and cleaned comments
print("\nOriginal and cleaned 'comments' (with timestamp sorting):")
print(df[[comments_col, 'cleaned_comments']].head(2))

print(f"\nMissing values in 'cleaned_comments': {df['cleaned_comments'].isnull().sum()}")

# Example to inspect a specific ticket's comments
first_comment_idx = df[df[comments_col].fillna('').str.len() > 0].index.min()
if pd.notna(first_comment_idx):
    print(f"\nExample of cleaned comments for a specific ticket (index {first_comment_idx}):")
    print("Original JSON string (first 500 chars):")
    print(str(df.loc[first_comment_idx, comments_col])[:500]) # Print a snippet to avoid huge output
    print("\nProcessed 'cleaned_comments':")
    print(df.loc[first_comment_idx, 'cleaned_comments'])
else:
    print("\nNo rows with comments found to display as an example.")



Original and cleaned 'comments' (with timestamp sorting):
                                            comments  \
0  [{"author": "Harish Kumar", "timestamp": "2025...   
1  [{"author": "Harish Kumar", "timestamp": "2025...   

                                    cleaned_comments  
0  Harish Kumar: {panel:bgColor=#deebff}\nOrigina...  
1  Harish Kumar: {panel:bgColor=#deebff}\nOrigina...  

Missing values in 'cleaned_comments': 0

Example of cleaned comments for a specific ticket (index 0):
Original JSON string (first 500 chars):
[{"author": "Harish Kumar", "timestamp": "2025-04-26T16:55:29.783+0530", "cleaned_body": "{panel:bgColor=#deebff}\nOriginal Author: Suresh Moturi\nPosted on: 2/21/2025, 7:14:30 PM\n{panel}\n\nFYI @Saurabh Verma", "mentions": []}, {"author": "Harish Kumar", "timestamp": "2025-04-26T16:55:30.459+0530", "cleaned_body": "{panel:bgColor=#deebff}\nOriginal Author: Nikhil Kailasanathan\nPosted on: 2/22/2025, 10:34:39 AM\n{panel}\n\n@Nitish Reddy Can you please share 

In [15]:
import re

# (Re-define the function if you're in a new cell, or modify the existing cell)
def strip_jira_markup(text):
    if not isinstance(text, str):
        return ""

    # {panel:...}content{panel} -> content
    text = re.sub(r'\{panel:[^}]*}(.*?)\{panel}', r'\1', text, flags=re.DOTALL)
    
    # {code[:lang]}content{code} -> content
    text = re.sub(r'\{code:[^}]*}(.*?)\{code}', r'\1', text, flags=re.DOTALL)
    text = re.sub(r'\{code}(.*?)\{code}', r'\1', text, flags=re.DOTALL)

    # {color:...}text{color} -> text
    text = re.sub(r'\{color:[^}]*}(.*?)\{color}', r'\1', text, flags=re.DOTALL)

    # Remove "Posted on: ..." lines (potentially left over from panel content)
    # This regex matches lines that start with "Posted on:" possibly with leading/trailing whitespace on the line itself.
    text = re.sub(r'^\s*Posted on:.*?\n', '', text, flags=re.MULTILINE)
    # If it might not end with a newline (e.g., last line of comment):
    text = re.sub(r'^\s*Posted on:.*?$', '', text, flags=re.MULTILINE)


    # Optional: Remove "Original Author: ..." lines if desired
    # text = re.sub(r'^\s*Original Author:.*?\n', '', text, flags=re.MULTILINE)
    # text = re.sub(r'^\s*Original Author:.*?$', '', text, flags=re.MULTILINE)

    # h1. to h6. headers
    text = re.sub(r'h[1-6]\.\s*', '', text)
    # *bold*
    text = re.sub(r'\*(.*?)\*', r'\1', text)
    # _italic_
    text = re.sub(r'_(.*?)_', r'\1', text)
    # +underline+
    text = re.sub(r'\+(.*?)\+', r'\1', text)
    # -strikethrough-
    text = re.sub(r'-(.*?)-', r'\1', text)
    # ??citation??
    text = re.sub(r'\?\?(.*?)\?\?', r'\1', text)
    # {{monospaced}}
    text = re.sub(r'\{\{(.*?)\}\}', r'\1', text)
    # bq. blockquote
    text = re.sub(r'bq\.\s+', '', text)
    # Links
    text = re.sub(r'\[([^|\]]+)\|[^\]]+\]', r'\1', text)
    text = re.sub(r'\[([^\]]+)\]', r'\1', text)
    # Images
    text = re.sub(r'!([^!]+)!', '', text)
    # Basic list markers
    text = re.sub(r'^\s*[\*#-]\s+', '', text, flags=re.MULTILINE)
    # noformat, quote
    text = re.sub(r'\{noformat\}(.*?)\{noformat\}', r'\1', text, flags=re.DOTALL)
    text = re.sub(r'\{quote\}(.*?)\{quote\}', r'\1', text, flags=re.DOTALL)
    
    return text

# --- You would then re-apply this updated function ---
# --- and the subsequent whitespace and case normalization steps ---

# print("Re-applying cleaning starting from Step 3 (Markup Removal)...")

# Step 3: Apply updated strip_jira_markup
# df['cleaned_summary'] = df[summary_col].fillna('').astype(str).apply(strip_jira_markup) # Re-start from original filled NAs if making big changes
# df['cleaned_description'] = df[description_col].fillna('').astype(str).apply(strip_jira_markup)
# df['cleaned_comments'] = df[comments_col].apply(parse_and_format_comments).apply(strip_jira_markup) # Re-parse comments then strip

# For simplicity if previous steps (1 & 2) are solid, you can just apply to the already partially cleaned columns:
df['cleaned_summary'] = df['cleaned_summary'].apply(strip_jira_markup)
df['cleaned_description'] = df['cleaned_description'].apply(strip_jira_markup)
df['cleaned_comments'] = df['cleaned_comments'].apply(strip_jira_markup)


# Step 4: Re-apply normalize_whitespace 
# (because removing lines might leave extra newlines that then need to be collapsed)
# (Assuming normalize_whitespace function is already defined in a previous cell)
df['cleaned_summary'] = df['cleaned_summary'].apply(normalize_whitespace)
df['cleaned_description'] = df['cleaned_description'].apply(normalize_whitespace)
df['cleaned_comments'] = df['cleaned_comments'].apply(normalize_whitespace)

# Step 5: Re-apply standardize_case
# (Assuming standardize_case function is already defined)
df['cleaned_summary'] = df['cleaned_summary'].apply(standardize_case)
df['cleaned_description'] = df['cleaned_description'].apply(standardize_case)
df['cleaned_comments'] = df['cleaned_comments'].apply(standardize_case)

print("\nText after UPDATED Jira markup stripping, whitespace, and case normalization:")
print("Cleaned Summary:")
print(df['cleaned_summary'].head(2).to_string())
print("\nCleaned Description:")
print(df['cleaned_description'].head(2).to_string())
print("\nCleaned Comments (example from first ticket with comments):")

first_comment_idx = df[df[comments_col].fillna('').str.len() > 0].index.min()
if pd.notna(first_comment_idx):
    print(f"(Ticket index {first_comment_idx})")
    print(df.loc[first_comment_idx, 'cleaned_comments'])
else:
    print("No comments found to display.")



Text after UPDATED Jira markup stripping, whitespace, and case normalization:
Cleaned Summary:
0    optum cleanup to fix issues due to incorrect s...
1    trigger event replay for city of tampa februar...

Cleaned Description:
0    due to incorrect config in the stride feature,...
1    for the promotions that were created post febr...

Cleaned Comments (example from first ticket with comments):
(Ticket index 0)
harish kumar: original author: suresh moturi posted on: 2/21/2025, 7:14:30 pm fyi @saurabh verma harish kumar: original author: nikhil kailasanathan posted on: 2/22/2025, 10:34:39 am @nitish reddy can you please share more details here? like promotion name, target event details, and impacted users if it is aware. harish kumar: original author: akhilesh baldi posted on: 2/22/2025, 11:23:12 am @nitish reddy @prateek yadav could you please help me harish kumar: original author: nitish reddy posted on: 2/22/2025, 11:23:49 am @akhilesh baldi i’m pulling the list of users, promotion 

In [10]:
import re

# --- Step 4: Normalize Whitespace ---

def normalize_whitespace(text):
    if not isinstance(text, str):
        return ""
    
    # Step 4.1: Replace multiple spaces, tabs, and newlines with a single space
    # \s+ matches one or more whitespace characters (space, tab, newline, etc.)
    text = re.sub(r'\s+', ' ', text)
    
    # Step 4.2: Trim leading and trailing whitespace
    text = text.strip()
    
    return text

# Apply to the relevant text columns
df['cleaned_summary'] = df['cleaned_summary'].apply(normalize_whitespace)
df['cleaned_description'] = df['cleaned_description'].apply(normalize_whitespace)
df['cleaned_comments'] = df['cleaned_comments'].apply(normalize_whitespace)

print("\nText after whitespace normalization (first 2 rows):")
print("Cleaned Summary:")
print(df['cleaned_summary'].head(2).to_string())
print("\nCleaned Description:")
print(df['cleaned_description'].head(2).to_string())
print("\nCleaned Comments (example from first ticket with comments):")

first_comment_idx = df[df[comments_col].fillna('').str.len() > 0].index.min()
if pd.notna(first_comment_idx):
    print(f"(Ticket index {first_comment_idx})")
    # To see the effect clearly, especially if there were many newlines in comments,
    # print the version before this step vs after, if you have it saved.
    # For now, just printing the current state.
    print(df.loc[first_comment_idx, 'cleaned_comments'])
else:
    print("No comments found to display.")



Text after whitespace normalization (first 2 rows):
Cleaned Summary:
0    Optum Cleanup to fix issues due to incorrect S...
1    Trigger Event Replay for City of Tampa Februar...

Cleaned Description:
0    Due to incorrect config in the Stride feature,...
1    For the promotions that were created post Febr...

Cleaned Comments (example from first ticket with comments):
(Ticket index 0)
Harish Kumar: Original Author: Suresh Moturi Posted on: 2/21/2025, 7:14:30 PM FYI @Saurabh Verma Harish Kumar: Original Author: Nikhil Kailasanathan Posted on: 2/22/2025, 10:34:39 AM @Nitish Reddy Can you please share more details here? like promotion name, target event details, and impacted users If it is aware. Harish Kumar: Original Author: Akhilesh Baldi Posted on: 2/22/2025, 11:23:12 AM @Nitish Reddy @Prateek Yadav could you please help me Harish Kumar: Original Author: Nitish Reddy Posted on: 2/22/2025, 11:23:49 AM @Akhilesh Baldi I’m pulling the list of users, promotion ids, target ids. Will add 

In [11]:
# --- Step 5: Standardize Case ---

def standardize_case(text):
    if not isinstance(text, str):
        return ""
    
    # Step 5.1: Convert text to lowercase
    text = text.lower()
    
    return text

# Apply to the relevant text columns
df['cleaned_summary'] = df['cleaned_summary'].apply(standardize_case)
df['cleaned_description'] = df['cleaned_description'].apply(standardize_case)
df['cleaned_comments'] = df['cleaned_comments'].apply(standardize_case)

print("\nText after case standardization (first 2 rows):")
print("Cleaned Summary:")
print(df['cleaned_summary'].head(2).to_string())
print("\nCleaned Description:")
print(df['cleaned_description'].head(2).to_string())
print("\nCleaned Comments (example from first ticket with comments):")

# Assuming 'comments_col' variable is still defined from previous steps
first_comment_idx = df[df[comments_col].fillna('').str.len() > 0].index.min()
if pd.notna(first_comment_idx):
    print(f"(Ticket index {first_comment_idx})")
    print(df.loc[first_comment_idx, 'cleaned_comments'])
else:
    print("No comments found to display.")



Text after case standardization (first 2 rows):
Cleaned Summary:
0    optum cleanup to fix issues due to incorrect s...
1    trigger event replay for city of tampa februar...

Cleaned Description:
0    due to incorrect config in the stride feature,...
1    for the promotions that were created post febr...

Cleaned Comments (example from first ticket with comments):
(Ticket index 0)
harish kumar: original author: suresh moturi posted on: 2/21/2025, 7:14:30 pm fyi @saurabh verma harish kumar: original author: nikhil kailasanathan posted on: 2/22/2025, 10:34:39 am @nitish reddy can you please share more details here? like promotion name, target event details, and impacted users if it is aware. harish kumar: original author: akhilesh baldi posted on: 2/22/2025, 11:23:12 am @nitish reddy @prateek yadav could you please help me harish kumar: original author: nitish reddy posted on: 2/22/2025, 11:23:49 am @akhilesh baldi i’m pulling the list of users, promotion ids, target ids. will add to t

In [12]:
df[['cleaned_summary', 'cleaned_description', 'cleaned_comments']].head(10)

,cleaned_summary,cleaned_description,cleaned_comments
0,optum cleanup to fix issues due to incorrect s...,"due to incorrect config in the stride feature,...",harish kumar: original author: suresh moturi p...
1,trigger event replay for city of tampa februar...,for the promotions that were created post febr...,harish kumar: original author: sourabh tiwari ...
2,optum - promotion rule not evaluating correctly,we have a case where a promotion rule appears ...,harish kumar: original author: bhagya yasam po...
3,request for access to dev console,"hi team, access to the dev console has been re...",harish kumar: original author: suresh moturi p...
4,access to aws workspace and api tester for ushc,"hi team, kindly provide access to the ushc aws...",harish kumar: original author: nihar jain post...
5,timezone execution,,
6,sandbox execution,,
7,extension/neo optimsation amj'25,,
8,null pointer exception in v2/usergroup2/member...,request details: url: http://internalintouchap...,
9,intouchapiv3 service pod down,"error errno: 111, code: econnrefused, syscall:...",
